# Aula 5 - Hipóteses mais complexas e regularização

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Hipóteses mais complexas
- 2) Regularização

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

____
____
____

_____

## 1) Hipóteses mais complexas

Muitas vezes, temos dados que simplesmente não se ajustam às hipóteses simples, lineares, que conhecemos até o momento.

Quando isso acontece, é muito provável que soframos **underfitting**, pois uma forma funcional demasiadamente simples de uma hipótese pode não ser capaz de capturar o comportamento de uma função teórica $\mathcal{F}$ mais complexa, conforme refletido pela amostra.

Nestes casos, a solução é simples: basta escolhermos hipóteses mais complexas!

Pra começar nosso estudo, vamos utilizar dados bem simples do próprio sklearn (submódulo [datasets](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets)):


In [ ]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 700, n_features = 1,
                       noise = 35, tail_strength = 50,
                       random_state = 42)

plt.scatter(X, y)
plt.show()

In [ ]:
X, y = make_regression(n_samples = 700, n_features = 1,
                       noise = 35, tail_strength = 50,
                       random_state = 42)

y = y**2

plt.scatter(X, y)
plt.show()

Podemos fazer uma regressão linear...

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ======================================

from sklearn.linear_model import LinearRegression

reg_lin = LinearRegression()

# ======================================

reg_lin.fit(X_train, y_train)

print(f"Intercepto (b0): {reg_lin.intercept_}")
print(f"Demais parâmetros (b1, ..., bn): {reg_lin.coef_}")

# ======================================

# como temos uma unica feature, dá pra plotar o modelo (hipótese final)
print("\nModelo treinado:")

x_plot = np.linspace(X.min(), X.max(), 1000)

y_plot = reg_lin.intercept_ + reg_lin.coef_[0]*x_plot

plt.plot(x_plot, y_plot, color="red")

# dados
plt.scatter(X, y)

plt.show()

# ======================================

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# predições de treino
y_pred_train = reg_lin.predict(X_train)

print("Métricas de treino:\n")
print(f"R^2: {r2_score(y_train, y_pred_train):.2f}")
print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")

# predições de teste
y_pred_test = reg_lin.predict(X_test)

print("\nMétricas de teste:\n")
print(f"R^2: {r2_score(y_test, y_pred_test):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")

Naturalmente, temos métricas bem ruins, dada a escolha ruim de hipótese!

Hipótese atual:

$$f_{h, \  \vec{b}}(x) = b_0 + b_1x$$

Vamos fazer algo melhor: como nossos dados são aproximadamente quadráticos, faria sentido escolher uma **hipótese quadrática**, não é mesmo? Isto é,

$$f_{h, \  \vec{b}}(x) = b_0 + b_1x + b_2x^2$$

E é aqui que entra um dos aspectos mais importantes de um modelo linear como a regressão linear: **o modelo é linear nos parâmetros, não necessariamente nas features!**

Ou seja, o termo quadrado que incluímos **pode ser considerado como uma nova feature linear**. Para ver isso, basta definir $z \equiv x^2$, que voltamos a ter uma hipótese linear, mas agora em duas variáveis:

$$f_{h, \  \vec{b}}(x, z) = b_0 + b_1x + b_2z$$

Ou seja, ainda temos uma regressão linear (múltipla, agora).

E isso é verdade para **qualquer** combinação de features que possamos criar!

Assim, para criarmos um modelo quadrático para nossos dados, bastaria criarmos uma nova feature $z = x^2$, e passar apenas esta nova feature para o  modelo de regressão linear **simples**. Isso equivale a usar uma hipótese $$f_{h, \  \vec{b}}(z) = b_0 + b_1z = b_0 + b_1x^2$$

Vejamos:

In [ ]:
# isso a feature z = x^2
# note: isso é um PRE PROCESSAMENTO DOS DADOS!!! nao to mexendo em NADA do estimador
Z = X**2

# ======================================

from sklearn.model_selection import train_test_split

# note que aqui tamos usando o Z ao inves do X
X_train, X_test, y_train, y_test = train_test_split(Z, y, test_size=0.2, random_state=42)

# ======================================

from sklearn.linear_model import LinearRegression

reg_lin = LinearRegression()

# ======================================

reg_lin.fit(X_train, y_train)

print(f"Intercepto (b0): {reg_lin.intercept_}")
print(f"Demais parâmetros (b1, ..., bn): {reg_lin.coef_}")

# ======================================

# como temos uma unica feature, dá pra plotar o modelo (hipótese final)
print("\nModelo treinado:")

x_plot = np.linspace(X.min(), X.max(), 1000)

y_plot = reg_lin.intercept_ + reg_lin.coef_[0]*(x_plot**2)

plt.plot(x_plot, y_plot, color="red")

# dados
plt.scatter(X, y)

plt.show()

# ======================================

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# predições de treino
y_pred_train = reg_lin.predict(X_train)

print("Métricas de treino:\n")
print(f"R^2: {r2_score(y_train, y_pred_train):.2f}")
print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")

# predições de teste
y_pred_test = reg_lin.predict(X_test)

print("\nMétricas de teste:\n")
print(f"R^2: {r2_score(y_test, y_pred_test):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")

Agora sim, um modelo beeem melhor!!

E se quisermos usar a hipótese quadrática mais completa, com ambos os termos linear e quadrático? (Isto é, $f_{h, \  \vec{b}}(x) = b_0 + b_1x + b_2x^2$)

Bem simples: basta passarmos as duas features pro sklearn:

In [ ]:
X_df = pd.DataFrame(X, columns=["X"])

In [ ]:
X_df

In [ ]:
X_df["Z"] = X_df["X"]**2

In [ ]:
X_df

In [ ]:
# ======================================

from sklearn.model_selection import train_test_split

# note que aqui tamos usando o X_df (com as duas variaveis) ao inves do X
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)

# ======================================

from sklearn.linear_model import LinearRegression

reg_lin = LinearRegression()

# ======================================

reg_lin.fit(X_train, y_train)

print(f"Intercepto (b0): {reg_lin.intercept_}")
print(f"Demais parâmetros (b1, ..., bn): {reg_lin.coef_}")

# ======================================

# como temos uma unica feature, dá pra plotar o modelo (hipótese final)
print("\nModelo treinado:")

x_plot = np.linspace(X.min(), X.max(), 1000)

y_plot = reg_lin.intercept_ + reg_lin.coef_[0]*(x_plot) + reg_lin.coef_[1]*(x_plot**2)

plt.plot(x_plot, y_plot, color="red")

# dados
plt.scatter(X, y)

plt.show()

# ======================================

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# predições de treino
y_pred_train = reg_lin.predict(X_train)

print("Métricas de treino:\n")
print(f"R^2: {r2_score(y_train, y_pred_train):.2f}")
print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")

# predições de teste
y_pred_test = reg_lin.predict(X_test)

print("\nMétricas de teste:\n")
print(f"R^2: {r2_score(y_test, y_pred_test):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")

No geral, dá pra ir aumentando a ordem dos polinomios criando features de ordem maior uma a uma:

In [ ]:
X_df["A"] = X_df["X"]**3
X_df["B"] = X_df["X"]**4

In [ ]:
X_df

In [ ]:
# ======================================

from sklearn.model_selection import train_test_split

# note que aqui tamos usando o X_df (com as duas variaveis) ao inves do X
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)

# ======================================

from sklearn.linear_model import LinearRegression

reg_lin = LinearRegression()

# ======================================

reg_lin.fit(X_train, y_train)

print(f"Intercepto (b0): {reg_lin.intercept_}")
print(f"Demais parâmetros (b1, ..., bn): {reg_lin.coef_}")

# ======================================

# como temos uma unica feature, dá pra plotar o modelo (hipótese final)
print("\nModelo treinado:")

x_plot = np.linspace(X.min(), X.max(), 1000)

y_plot = (reg_lin.intercept_ + reg_lin.coef_[0]*(x_plot)
          + reg_lin.coef_[1]*(x_plot**2) + reg_lin.coef_[2]*(x_plot**3)
          + reg_lin.coef_[3]*(x_plot**4))

plt.plot(x_plot, y_plot, color="red")

# dados
plt.scatter(X, y)

plt.show()

# ======================================

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# predições de treino
y_pred_train = reg_lin.predict(X_train)

print("Métricas de treino:\n")
print(f"R^2: {r2_score(y_train, y_pred_train):.2f}")
print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")

# predições de teste
y_pred_test = reg_lin.predict(X_test)

print("\nMétricas de teste:\n")
print(f"R^2: {r2_score(y_test, y_pred_test):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")

O procedimento acima é bem manual. Pra nossa sorte, o sklearn existe, e uma de suas muitas ferramentas especiais para machine learning (no caso, pré-processamento) é o [polynomial features](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html), que permite a criação de toda as combinações polinomiais de features automaticamente!

O PolynomialFeatures é nosso primeiro exemplo de **transformer** do sklearn - um método cujo objetivo é aplicar alguma **transformação** aos dados. Veremos vários outros exemplos de transformers durante o curso.

Em particular, todos os transformers se comportam como se fossem "estimadores", no sentido de que eles devem 
ser "ajustados" aos dados -- por isso, eles também têm o método `.fit()` -- que ajusta o transformer aos dados; além do método `.transform()`, que efetivamente transforma os dados. Existe também o `.fit_transform()`, que faz as duas coisas ao mesmo tempo -- mas vamos evitar de usá-lo, por motivos que ficarão claros no futuro próximo.

Lembre-se de fitar o transformador sempre nos dados de treino, apenas! Neste caso, não faz muita diferença, mas, para nos acostumarmos a isso, vamos fazer aqui também!

In [ ]:
X, y = make_regression(n_samples = 700, n_features = 1,
                       noise = 35, tail_strength = 50,
                       random_state = 42)

y = y**2

plt.scatter(X, y)
plt.show()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures(degree=2, include_bias=False)

# pra gente se acostumar: fit só em dados de treino!!!!
pf.fit(X_train)

In [ ]:
vars(pf)

In [ ]:
pf.n_features_in_

In [ ]:
pf.n_output_features_

In [ ]:
# das duas features que teremos depois da transformação
# a primeira tem grau 1
# a segunda tem grau 2

pf.powers_

In [ ]:
X_train_transf = pf.transform(X_train)
X_test_transf = pf.transform(X_test)

In [ ]:
X_train_transf

In [ ]:
X_test_transf

Tudo numa unica célula:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ======================================
# passo adicional: criando features polinomiais
# pra deixar a hipotese mais complexa (regressão linear em espaço polinomial)

from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures(degree=2, include_bias=False)

# pra gente se acostumar: fit só em dados de treino!!!!
pf.fit(X_train)

print(f"Número original de features: {pf.n_features_in_}")
print(f"Número de features no espaço transformado: {pf.n_output_features_}\n\n")

# redefinindo as features de treino e de teste
X_train = pf.transform(X_train)
X_test = pf.transform(X_test)

# ======================================

from sklearn.linear_model import LinearRegression

reg_lin = LinearRegression()

# ======================================

reg_lin.fit(X_train, y_train)

print(f"Intercepto (b0): {reg_lin.intercept_}")
print(f"Demais parâmetros (b1, ..., bn): {reg_lin.coef_}")

# ======================================

# como temos uma unica feature, dá pra plotar o modelo (hipótese final)
print("\nModelo treinado:")

x_plot = np.linspace(X.min(), X.max(), 1000)

# pra plotar a hipotese automaticamente
# y = bo + b1*x^1 + b2*x^2 + .... + bn*x^n

# y_plot = reg_lin.intercept_

# for i in range(len(reg_lin.coef_)):
    
#     y_plot = y_plot + reg_lin.coef_[i]*(x_plot**(i+1))

y_plot = reg_lin.intercept_

for n, b_n in enumerate(reg_lin.coef_):
    
    y_plot = y_plot + b_n*(x_plot**(n+1))

plt.plot(x_plot, y_plot, color="red")

# dados
plt.scatter(X, y)

plt.show()

# ======================================

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# predições de treino
y_pred_train = reg_lin.predict(X_train)

print("Métricas de treino:\n")
print(f"R^2: {r2_score(y_train, y_pred_train):.2f}")
print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")

# predições de teste
y_pred_test = reg_lin.predict(X_test)

print("\nMétricas de teste:\n")
print(f"R^2: {r2_score(y_test, y_pred_test):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")

### Então podemos pensar que quanto mais features melhor será o nosso modelo?

**Maldição da dimensionalidade**

Este fenômeno afirma que com um número fixo de amostras de treinamento, o poder preditivo médio (esperado) de um classificador ou regressor aumenta primeiro à medida que o número de dimensões ou características utilizadas aumenta, mas além de uma certa dimensionalidade, começa a deteriorar-se em vez de melhorar de forma constante. Este aumento na dimensionalidade do problema pode se refletir no overfitting de um modelo. Vamos ver isso claramente?

In [ ]:
# prototipo pra salvar resultados

# resultados = {"num_features" : [1, 2, 3],
#               "mae_train" : [425432, 454253324, 435645654676798],
#               "mae_test" : [42424325, 5434234253, 435645654676798]}

# pd.DataFrame(resultados)

Prototipo do que fizemos abaixo

In [ ]:
resultados = {"num_features" : [],
              "mae_train" : [],
              "mae_test" : []}

In [ ]:
resultados["num_features"].append(4)
resultados["mae_train"].append(5453435)
resultados["mae_test"].append(45345345)

In [ ]:
resultados

Agora sim:

In [ ]:
# dicionario de resultados do experimento
resultados = {"num_features" : [],
              "mae_train" : [],
              "mae_test" : []}

for grau in range(1, 16):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # ======================================

    pf = PolynomialFeatures(degree=grau, include_bias=False)

    pf.fit(X_train)

    print(f"Número original de features: {pf.n_features_in_}")
    print(f"Número de features no espaço transformado: {pf.n_output_features_}\n\n")

    # redefinindo as features de treino e de teste
    X_train = pf.transform(X_train)
    X_test = pf.transform(X_test)

    # ======================================

    reg_lin = LinearRegression()

    reg_lin.fit(X_train, y_train)

    # ======================================

    # predições de treino
    y_pred_train = reg_lin.predict(X_train)

    print("Métricas de treino:\n")
    print(f"R^2: {r2_score(y_train, y_pred_train):.2f}")
    print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")

    # predições de teste
    y_pred_test = reg_lin.predict(X_test)

    print("\nMétricas de teste:\n")
    print(f"R^2: {r2_score(y_test, y_pred_test):.2f}")
    print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")
    
    print()
    print("#"*80)
    print()
    
    # ======================================
    
    resultados["num_features"].append(pf.n_output_features_)
    resultados["mae_train"].append(mean_absolute_error(y_train, y_pred_train))
    resultados["mae_test"].append(mean_absolute_error(y_test, y_pred_test))

In [ ]:
# dataframe de resultados do experimento

df_resultados = pd.DataFrame(resultados)

In [ ]:
df_resultados

In [ ]:
df_resultados.describe()

In [ ]:
plt.title("Tradeoff viés-variância")

plt.plot(df_resultados["num_features"], df_resultados["mae_train"], label="MAE train")
plt.plot(df_resultados["num_features"], df_resultados["mae_test"], label="MAE test")

plt.legend()
plt.show()

In [ ]:
plt.title("Tradeoff viés-variância")

plt.plot(df_resultados[:11]["num_features"], df_resultados[:11]["mae_train"], label="MAE train")
plt.plot(df_resultados[:11]["num_features"], df_resultados[:11]["mae_test"], label="MAE test")

plt.legend()
plt.show()

_____________
_____________
_____________

Agora que já entendemos a técnica em um dataset bem simples, vamos voltar pra um dataset real!

Vamos voltar pros dados da precificação de casas -- ali, o poly_features se mostrará ainda mais útil!

In [ ]:
df = pd.read_csv("house_prices.csv")

# incluindo apenas features numericas, jogando fora os NaNs
df = df.select_dtypes(include=np.number).dropna()

X = df.drop(columns=["SalePrice", "Id"])
y = df["SalePrice"]

# ======================================

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ======================================
# passo adicional: criando features polinomiais
# pra deixar a hipotese mais complexa (regressão linear em espaço polinomial)

from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures(degree=1, include_bias=False)

# pra gente se acostumar: fit só em dados de treino!!!!
pf.fit(X_train)

print(f"Número original de features: {pf.n_features_in_}")
print(f"Número de features no espaço transformado: {pf.n_output_features_}\n\n")

# redefinindo as features de treino e de teste
X_train = pf.transform(X_train)
X_test = pf.transform(X_test)

# ======================================

from sklearn.linear_model import LinearRegression

reg_lin = LinearRegression()

# ======================================

reg_lin.fit(X_train, y_train)

print(f"Intercepto (b0): {reg_lin.intercept_}")
print(f"Demais parâmetros (b1, ..., bn): {reg_lin.coef_}")

# ======================================

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# predições de treino
y_pred_train = reg_lin.predict(X_train)

print("\nMétricas de treino:\n")
print(f"R^2: {r2_score(y_train, y_pred_train):.2f}")
print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")

# predições de teste
y_pred_test = reg_lin.predict(X_test)

print("\nMétricas de teste:\n")
print(f"R^2: {r2_score(y_test, y_pred_test):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")

In [ ]:
df = pd.read_csv("../datasets/house_prices.csv")

# incluindo apenas features numericas, jogando fora os NaNs
df = df.select_dtypes(include=np.number).dropna()

X = df.drop(columns=["SalePrice", "Id"])
y = df["SalePrice"]

# ======================================

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ======================================
# passo adicional: criando features polinomiais
# pra deixar a hipotese mais complexa (regressão linear em espaço polinomial)

from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures(degree=2, include_bias=False)

# pra gente se acostumar: fit só em dados de treino!!!!
pf.fit(X_train)

print(f"Número original de features: {pf.n_features_in_}")
print(f"Número de features no espaço transformado: {pf.n_output_features_}\n\n")

# redefinindo as features de treino e de teste
X_train = pf.transform(X_train)
X_test = pf.transform(X_test)

# ======================================

from sklearn.linear_model import LinearRegression

reg_lin = LinearRegression()

# ======================================

reg_lin.fit(X_train, y_train)

print(f"Intercepto (b0): {reg_lin.intercept_}")
print(f"Demais parâmetros (b1, ..., bn): {reg_lin.coef_}")

# ======================================

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# predições de treino
y_pred_train = reg_lin.predict(X_train)

print("\nMétricas de treino:\n")
print(f"R^2: {r2_score(y_train, y_pred_train):.2f}")
print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")

# predições de teste
y_pred_test = reg_lin.predict(X_test)

print("\nMétricas de teste:\n")
print(f"R^2: {r2_score(y_test, y_pred_test):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")

Esete ultimo modelo tinha muuuuuito mais parametros que observações, portanto, aprendeu perfeitamente até mesmo os ruidos da base de treino!!

Com quantas features o modelo final foi construído?

In [ ]:
pf.n_output_features_

Nossa hipótese é:

$$ f_{H, \vec{b}}(\vec{x}) = b_0 + b_1x_1 + b_2x_2 + \cdots + b_{702} x_{702}$$

Ou seja, temos um modelo **com muitos parâmetros**, ou seja, **muito complexo!**

Com tantos parâmetros assim, há muitos **graus de liberdade** pra que a hipótese se ajuste até às particularidades da base de treino... 

O resultado é evidente: temos um modelo altamente **overfitado**, dado o número enorme de features após o transformer -- e isso porque estamos utilizando apenas features quadráticas, imagine se tivéssemos usado features de grau maior!

É de se imaginar que muitas destas features não deveriam estar aí, não é mesmo?

Oras, uma forma interessante de eliminar features é fazendo o que chamamos de **feature selection**.

A ideia é a seguinte: gostaríamos sim de introduzir features quadráticas, aumentando um pouco a complexidade da hipótese, **mas não tanto!**. 

E é isso que conseguiremos fazer com as técnicas de **regularização**.

Antes, vamos chutar mais o balde...

In [ ]:
# uma saida, é limitar a transformação

df = pd.read_csv("../datasets/house_prices.csv")

# incluindo apenas features numericas, jogando fora os NaNs
df = df.select_dtypes(include=np.number).dropna()

X = df.drop(columns=["SalePrice", "Id"])
y = df["SalePrice"]

# ======================================

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ======================================
# passo adicional: criando features polinomiais
# pra deixar a hipotese mais complexa (regressão linear em espaço polinomial)

from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)

# pra gente se acostumar: fit só em dados de treino!!!!
pf.fit(X_train)

print(f"Número original de features: {pf.n_features_in_}")
print(f"Número de features no espaço transformado: {pf.n_output_features_}\n\n")

# redefinindo as features de treino e de teste
X_train = pf.transform(X_train)
X_test = pf.transform(X_test)

# ======================================

from sklearn.linear_model import LinearRegression

reg_lin = LinearRegression()

# ======================================

reg_lin.fit(X_train, y_train)

print(f"Intercepto (b0): {reg_lin.intercept_}")
print(f"Demais parâmetros (b1, ..., bn): {reg_lin.coef_}")

# ======================================

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# predições de treino
y_pred_train = reg_lin.predict(X_train)

print("\nMétricas de treino:\n")
print(f"R^2: {r2_score(y_train, y_pred_train):.2f}")
print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")

# predições de teste
y_pred_test = reg_lin.predict(X_test)

print("\nMétricas de teste:\n")
print(f"R^2: {r2_score(y_test, y_pred_test):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")

In [ ]:
df = pd.read_csv("../datasets/house_prices.csv")

# incluindo apenas features numericas, jogando fora os NaNs
df = df.select_dtypes(include=np.number).dropna()

X = df.drop(columns=["SalePrice", "Id"])
y = df["SalePrice"]

# ======================================

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ======================================
# passo adicional: criando features polinomiais
# pra deixar a hipotese mais complexa (regressão linear em espaço polinomial)

from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures(degree=3, include_bias=False)

# pra gente se acostumar: fit só em dados de treino!!!!
pf.fit(X_train)

print(f"Número original de features: {pf.n_features_in_}")
print(f"Número de features no espaço transformado: {pf.n_output_features_}\n\n")

# redefinindo as features de treino e de teste
X_train = pf.transform(X_train)
X_test = pf.transform(X_test)

# ======================================

from sklearn.linear_model import LinearRegression

reg_lin = LinearRegression()

# ======================================

reg_lin.fit(X_train, y_train)

print(f"Intercepto (b0): {reg_lin.intercept_}")
print(f"Demais parâmetros (b1, ..., bn): {reg_lin.coef_}")

# ======================================

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# predições de treino
y_pred_train = reg_lin.predict(X_train)

print("\nMétricas de treino:\n")
print(f"R^2: {r2_score(y_train, y_pred_train):.2f}")
print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")

# predições de teste
y_pred_test = reg_lin.predict(X_test)

print("\nMétricas de teste:\n")
print(f"R^2: {r2_score(y_test, y_pred_test):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")

O que podemos dizer sobre este modelo?

**Claro overfitting!**

____
____
____

Pra nosso código ficar mais orgamizado, podemos fazer uma função para a modelagem (depois vcs podem refazer os passos acima com a função, ajuda a organizar o código!)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
    
def poly_reg(X, y, degree):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # ======================================
    # passo adicional: criando features polinomiais
    # pra deixar a hipotese mais complexa (regressão linear em espaço polinomial)

    pf = PolynomialFeatures(degree=degree, include_bias=False)

    # pra gente se acostumar: fit só em dados de treino!!!!
    pf.fit(X_train)

    print(f"Número original de features: {pf.n_features_in_}")
    print(f"Número de features no espaço transformado: {pf.n_output_features_}")

    # redefinindo as features de treino e de teste
    X_train = pf.transform(X_train)
    X_test = pf.transform(X_test)

    # ======================================

    reg_lin = LinearRegression()

    reg_lin.fit(X_train, y_train)

    # ======================================

    # predições de treino
    y_pred_train = reg_lin.predict(X_train)

    print("\nMétricas de treino:\n")
    print(f"R^2: {r2_score(y_train, y_pred_train):.2f}")
    print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")

    # predições de teste
    y_pred_test = reg_lin.predict(X_test)

    print("\nMétricas de teste:\n")
    print(f"R^2: {r2_score(y_test, y_pred_test):.2f}")
    print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")

In [ ]:
df = pd.read_csv("../datasets/house_prices.csv")
df = df.select_dtypes(include=np.number).dropna()

X = df.drop(columns=["SalePrice", "Id"])
y = df["SalePrice"]

In [ ]:
poly_reg(X, y, degree=1)

In [ ]:
poly_reg(X, y, degree=2)

In [ ]:
poly_reg(X, y, degree=3)

_____

## 2) Regularização

Neste ponto, é muito importante que falemos sobre **regularização**.

O objetivo da regularização é **diminuir a complexidade** de modelos, de modo a evitar que particularidades da base de treino (ruídos) sejam aprendidos (ou seja, evitar overfitting!)

Uma outra forma de enxergar regularização: **diminuição do espaço de hipóteses!**

<img src=https://curso-r.github.io/main-intro-ml/slides/static/img/erro_treino_erro_teste.png width=500>

Regularização: problema de otimização VINCULADO!! ou seja, com restrições.

problema de otimização: otimização da função de custo, que é o objetivo da aprendizagem, pra determinar o $\hat{\vec{b}}$

restrições: é o que determina se temos L1 (lasso) ou L2 (ridge)

### Regressão linear (sem regularização)

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/5408b0a7-85f3-4824-ad68-44867121ecb9.png width=800>

### L1 (Lasso)

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/acabe9da-07ba-4337-b467-dd2701a40cc8.png width=900>

### L2 (Ridge)

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/46eda310-fb2f-498b-b455-593183de1dd7.png width=900>

Para saber como relacionar $t$ com $\lambda$, veja [este post](https://stats.stackexchange.com/questions/259177/expressing-the-lasso-regression-constraint-via-the-penalty-parameter) ou então [este](https://stats.stackexchange.com/questions/90648/kkt-versus-unconstrained-formulation-of-lasso-regression) -- discussões bem matemáticas!

Observações importantes:

- $\lambda$ é um parâmetro que controla a "força" da regularização<br><br>
- **L1 pode zerar coeficientes** - faz feature selection<br><br>
- **L2 apenas diminui o tamanho de coeficientes** - não faz feature selection<br><br>

<img src=https://ugc.futurelearn.com/uploads/assets/2b/fe/2bfe399e-503e-4eae-9138-a3d7da738713.png width=900>



Geometricamente:

<img src=https://www.astroml.org/_images/fig_lasso_ridge_1.png width=800>

No sklearn, é possível fazer um modelo de regressão linear regularizado facilmente com as classes respectivas:

- [Regularização L2/Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge)

- [Regularização L1/Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso)

Há, no sklearn, também uma implementação para um tipo de regularização conhecida como **Elastic Net**:

<img src=https://miro.medium.com/max/761/1*nrWncnoJ4V_BkzEf1pd4MA.png width=900>

A classe se chama [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet)



Vamos utilizar regularização no dataset das casas, juntamente com as features polinomiais:

> **IMPORTANTE**: como os métodos de regularização são baseados na norma do vetor de parâmetros, é muito importante que as features sejam escaladas para que os métodos funcionem bem!

Isso porque a escala das features irá influenciar a regularização aplicada ao parâmetro respectivo!

Para eliminar este efeito, escalar os dados é muito importante!

Vamos visualizar concretamente como a regularização de fato simplifica a hipótese! Pra isso, considere os pontos a seguir:

In [ ]:
np.random.seed(42)
ruido = np.random.normal(0, 1, 10)

In [ ]:
ruido.reshape(-1, 1)

In [ ]:
y

In [ ]:
X = np.arange(10)
y = X**2

np.random.seed(42)
ruido = np.random.normal(0, 3, 10)
y = y + ruido

# isso é só pra poder treinar o modelo com 1 feature
X = X.reshape(-1, 1)

x_plot = np.linspace(0, 10, 1000)
y_plot = x_plot**2

plt.scatter(X, y)

plt.plot(x_plot, y_plot, color="r")

plt.show()

In [ ]:
plt.scatter(X, y)
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def calc_y_plot(estimator, x_plot):

    y_plot = estimator.intercept_

    for n, b_n in enumerate(estimator.coef_):

        y_plot = y_plot + b_n*(x_plot**(n+1))
    
    return y_plot

In [ ]:
X

In [ ]:
pf = PolynomialFeatures(degree=2, include_bias=False).fit(X)

X_transf = pf.transform(X)

X_transf

In [ ]:
mms = MinMaxScaler().fit(X_transf)

X_transf =  mms.transform(X_transf)

X_transf

In [ ]:
def reg_poly_plot(X, y, degree):
    
    # nestes caso APENAS, nao avaliaremos os modelos. Só queremos visualizar

    pf = PolynomialFeatures(degree=degree, include_bias=False).fit(X)

    X_transf = pf.transform(X)
    
    print(f"Número original de features: {pf.n_features_in_}")
    print(f"Número de features no espaço transformado: {pf.n_output_features_}")

    # =====================================

    mms = MinMaxScaler().fit(X_transf)

    X_transf =  mms.transform(X_transf)

    # =====================================

    reg_lin = LinearRegression().fit(X_transf, y)

    # =====================================

    print("\nModelo treinado:")

    x_plot = np.linspace(X_transf[:, 0].min(), X_transf[:, 0].max(), 1000)

    y_plot = calc_y_plot(reg_lin, x_plot)

    plt.scatter(X_transf[:, 0], y)

    plt.plot(x_plot, y_plot, color="r")
    plt.show()

In [ ]:
for degree in range(1, 11):
    
    reg_poly_plot(X, y, degree)
    
    print("#"*80)
    print()

Agora, vamos regularizar!

In [ ]:
from sklearn.linear_model import Ridge, Lasso

def reg_poly_regularized_plot(X, y, degree):
    
    # nestes caso APENAS, nao avaliaremos os modelos. Só queremos visualizar

    pf = PolynomialFeatures(degree=degree, include_bias=False).fit(X)

    X_transf = pf.transform(X)
    
    print(f"Número original de features: {pf.n_features_in_}")
    print(f"Número de features no espaço transformado: {pf.n_output_features_}")

    # =====================================

    mms = MinMaxScaler().fit(X_transf)

    X_transf =  mms.transform(X_transf)

    # =====================================

    reg_lin = LinearRegression().fit(X_transf, y)
    
    reg_l1 = Lasso(alpha=1).fit(X_transf, y)
    
    reg_l2 = Ridge(alpha=1).fit(X_transf, y)

    # =====================================

    print("\nModelo treinado:")
    
    plt.scatter(X_transf[:, 0], y)

    x_plot = np.linspace(X_transf[:, 0].min(), X_transf[:, 0].max(), 1000)

    y_plot_reg_lin = calc_y_plot(reg_lin, x_plot)
    plt.plot(x_plot, y_plot_reg_lin, color="r", label="rl", ls=":")
    
    y_plot_reg_l1 = calc_y_plot(reg_l1, x_plot)
    plt.plot(x_plot, y_plot_reg_l1, color="orange", label="L1")
    
    y_plot_reg_l2 = calc_y_plot(reg_l2, x_plot)
    plt.plot(x_plot, y_plot_reg_l2, color="green", label="L2")
    
    plt.legend()
    plt.show()

In [ ]:
from sklearn.linear_model import Ridge, Lasso

# pra ficar mais fácil de ver os parâmetros, vamos fixar três casas decimais
np.set_printoptions(formatter={'float': lambda x: "{:.3f}".format(x)})

def reg_poly_regularized_plot2(X, y, degree):
    
    # nestes caso APENAS, nao avaliaremos os modelos. Só queremos visualizar

    pf = PolynomialFeatures(degree=degree, include_bias=False).fit(X)

    X_transf = pf.transform(X)
    
    print(f"Número original de features: {pf.n_features_in_}")
    print(f"Número de features no espaço transformado: {pf.n_output_features_}")

    # =====================================

    mms = MinMaxScaler().fit(X_transf)

    X_transf =  mms.transform(X_transf)

    # =====================================

    reg_lin = LinearRegression().fit(X_transf, y)
    
    reg_l1 = Lasso(alpha=1).fit(X_transf, y)
    
    reg_l2 = Ridge(alpha=1).fit(X_transf, y)
    
    print(f"\nParâmetros modelo não regularizado:\n{reg_lin.intercept_:.3f}\n{reg_lin.coef_}")
    print(f"\nParâmetros modelo com L1 (Lasso):\n{reg_l1.intercept_:.3f}\n{reg_l1.coef_}")
    print(f"\nParâmetros modelo com L2 (Ridge):\n{reg_l2.intercept_:.3f}\n{reg_l2.coef_}")

    # =====================================

    print("\nModelo treinado:")
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    
    axs[0].scatter(X_transf[:, 0], y)
    axs[1].scatter(X_transf[:, 0], y)

    x_plot = np.linspace(X_transf[:, 0].min(), X_transf[:, 0].max(), 1000)

    y_plot_reg_lin = calc_y_plot(reg_lin, x_plot)
    axs[0].plot(x_plot, y_plot_reg_lin, color="r", label="RL", ls=":")
    axs[1].plot(x_plot, y_plot_reg_lin, color="r", label="RL", ls=":")
    
    y_plot_reg_l1 = calc_y_plot(reg_l1, x_plot)
    axs[0].plot(x_plot, y_plot_reg_l1, color="orange", label="L1")
    
    y_plot_reg_l2 = calc_y_plot(reg_l2, x_plot)
    axs[1].plot(x_plot, y_plot_reg_l2, color="green", label="L2")
    
    axs[0].legend()
    axs[1].legend()
    plt.show()

In [ ]:
for degree in range(1, 11):
    
    reg_poly_regularized_plot2(X, y, degree)
    
    print("#"*80)
    print()

In [ ]:
for degree in range(1, 11):
    
    reg_poly_regularized_plot(X, y, degree)
    
    print("#"*80)
    print()

**Lição de casa**: altere a função acima pra ter mais um argumento: alpha.

Daí, varie também o alpha (força de regularização).

_________
Vamos agora voltar pro dataset de precificação de casas:

______________
______________
______________

Comentário: discutimos já que é importante escalar as features quando formos usar regularização.

Mas, é muito importante que o scaling dos dados seja O ÚLTIMO PASSO!

Ou seja, se também quisermos fazer o polynomial features, temos que fazer ANTES do scaler. 

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
print(X)
print()

pf = PolynomialFeatures(degree=3, include_bias=False).fit(X)
X_transf = pf.transform(X)

print(X_transf)
print()

ss = StandardScaler().fit(X_transf)
X_transf = ss.transform(X_transf)

print(X_transf)

In [ ]:
print(X)
print()

ss = StandardScaler().fit(X)
X_transf = ss.transform(X)

print(X_transf)
print()

pf = PolynomialFeatures(degree=3, include_bias=False).fit(X_transf)
X_transf = pf.transform(X_transf)

print(X_transf)

______________
______________
______________

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
def poly_regularized_reg(X, y, degree, 
                         type_regularization=None, alpha=1, l1_ratio=0.5, 
                         iter_max=1000):
    '''
    - type_regularization (str): opções de regularização: ["l1", "l2", "en", None]
    '''

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # ======================================
 
    pf = PolynomialFeatures(degree=degree, include_bias=False).fit(X_train)

    # redefinindo as features de treino e de teste
    X_train = pf.transform(X_train)
    X_test = pf.transform(X_test)
    
    print(f"Número original de features: {pf.n_features_in_}")
    print(f"Número de features no espaço transformado: {pf.n_output_features_}")

    # ======================================
    # normalização dos dados - MUITO importante quando há regularização!!
    # e é o passo imediatamente antes de treinar os modelos
    
    mms = MinMaxScaler().fit(X_train)
    
    X_train = mms.transform(X_train)
    X_test = mms.transform(X_test)
    
    # ======================================

    if type_regularization == "l1":
        
        model = Lasso(alpha=alpha, max_iter=iter_max).fit(X_train, y_train)
        
    elif type_regularization == "l2":
        
        model = Ridge(alpha=alpha, max_iter=iter_max).fit(X_train, y_train)
        
    elif type_regularization == "en":
        
        model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=iter_max).fit(X_train, y_train)
        
    elif type_regularization == None:
    
        model = LinearRegression().fit(X_train, y_train)
        
    else:
        
        list_opcoes = ["l1", "l2", "en", None]
        raise ValueError(f"Opção de regularização indisponível!\nOpções aceitas: {list_opcoes}")


    # ======================================

    # predições de treino
    y_pred_train = model.predict(X_train)

    print("\nMétricas de treino:\n")
    print(f"R^2: {r2_score(y_train, y_pred_train):.2f}")
    print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")

    # predições de teste
    y_pred_test = model.predict(X_test)

    print("\nMétricas de teste:\n")
    print(f"R^2: {r2_score(y_test, y_pred_test):.2f}")
    print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}") 

In [ ]:
df = pd.read_csv("../datasets/house_prices.csv")
df = df.select_dtypes(include=np.number).dropna()

X = df.drop(columns=["SalePrice", "Id"])
y = df["SalePrice"]

In [ ]:
# regressão linear, sem regularização
# exatamente o que fizemos na primeira aula!
poly_regularized_reg(X, y, degree=1, type_regularization=None)

In [ ]:
poly_regularized_reg(X, y, degree=2, type_regularization=None)

___________

**Benchmark** (modelo linear que conseguíamos fazer antes dessa aula)

Métricas de treino:

R^2: 0.81<br>
MAE: 22286.55<br>
RMSE: 35650.58<br>

Métricas de teste:

R^2: 0.80<br>
MAE: 23662.02<br>
RMSE: 39859.00<br>

### L1

In [ ]:
poly_regularized_reg(X, y, degree=2, type_regularization="l1", alpha=100, iter_max=2000)

In [ ]:
## ESSE FOI O CAMPEÃO (por enquanto, rs)

poly_regularized_reg(X, y, degree=3, type_regularization="l1", alpha=100, iter_max=2000)

### L2

In [ ]:
poly_regularized_reg(X, y, degree=2, type_regularization="l2", alpha=200, iter_max=2000)

In [ ]:
poly_regularized_reg(X, y, degree=2, type_regularization="l2", alpha=50, iter_max=2000)

In [ ]:
poly_regularized_reg(X, y, degree=3, type_regularization="l2", alpha=50, iter_max=2000)

### Elastic Net

In [ ]:
poly_regularized_reg(X, y, degree=2, type_regularization="en", 
                     alpha=1, l1_ratio=0.5,
                     iter_max=2000)

In [ ]:
poly_regularized_reg(X, y, degree=2, type_regularization="en", 
                     alpha=10, l1_ratio=0.8,
                     iter_max=2000)

In [ ]:
poly_regularized_reg(X, y, degree=3, type_regularization="en", 
                     alpha=100, l1_ratio=0.8,
                     iter_max=2000)

_____

Uma pergunta importante é: **como selecionar um valor adequado para os parâmetros de regularização?**

Naturalmente, este é um hiperarâmetro bastante importante, dado que ele controla a "força" da regularização a ser aplicada.

E, no caso do elastic net, o parâmetro de mistura também é muito relevante!

Uma abordagem para a escolha de valores adequados de hiperparâmetros (processo chamado de **hyperparameter tuning**) é testar exaustivamente vários valores com o processo de **validação cruzada**, de modo a encontrarmos os melhores valores (e/ou combinação de valores) de hiperparâmetros.

Antes de nos aprofundarmos no processo de tuning, vamos entender melhor o que é a validação cruzada!

Para isso, veja o notebook da próxima aula! ;)